In [0]:
%pip install databricks-sdk>=0.57.0
dbutils.library.restartPython()

In [0]:
%run ../00.set_variables

In [0]:
import yaml

#create app.yaml
data = {
    "command": [
        "streamlit",
        "run",
        "app.py"
    ],
    "env": [
        {
            "name": "DATABRICKS_WAREHOUSE_ID",
            "valueFrom": "sql-warehouse"
        },
        {
            "name": "STREAMLIT_BROWSER_GATHER_USAGE_STATS",
            "value": "false"
        },
        {
            "name": "TABLE_NAME",
            "value": f"{catalog_name}.{schema_name}.gold_iop_features_version_demo"
        },
        {
            "name": "FE_ENDPOINT_NAME",
            "value": f"{schema_name}-fe_model-serving-endpoint"
        },
        {
            "name": "SI_ENDPOINT_NAME",
            "value": f"{schema_name}-si_model-serving-endpoint"
        }
    ]
}

with open("../src/app.yaml", "w") as f:
    yaml.dump(data, f, sort_keys=False)

In [0]:
#get default warehouse
# Import necessary modules from the Databricks SDK
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import sql

# --- Configuration and Initialization ---
# Initialize the Databricks WorkspaceClient.
# If running this code directly in a Databricks notebook,
# the client will automatically pick up authentication from the notebook's context.
# If running locally, ensure your Databricks authentication is configured (e.g., via
# DATABRICKS_HOST and DATABRICKS_TOKEN environment variables, or ~/.databrickscfg).
w = WorkspaceClient()

print("Attempting to find the first available SQL warehouse ID...")

try:
    # 1. List all SQL warehouses
    # The list() method returns an iterator of sql.Warehouse objects.
    # We convert it to a list to enable sorting.
    all_warehouses = list(w.warehouses.list())

    if not all_warehouses:
        print("No SQL warehouses found in this workspace.")
        print("Please ensure you have at least one SQL warehouse created.")
    else:
        # 2. Separate running and stopped warehouses
        running_warehouses = []
        stopped_warehouses = []
        for wh in all_warehouses:
            if wh.state == sql.State.RUNNING:
                running_warehouses.append(wh)
            else:
                stopped_warehouses.append(wh)

        # 3. Sort both lists alphabetically by name
        running_warehouses.sort(key=lambda wh: wh.name.lower())
        stopped_warehouses.sort(key=lambda wh: wh.name.lower())

        first_available_warehouse = None

        if running_warehouses:
            # Prioritize running warehouses
            first_available_warehouse = running_warehouses[0]
            print("Found a running SQL warehouse.")
        elif stopped_warehouses:
            # If no running, take the first stopped one
            first_available_warehouse = stopped_warehouses[0]
            print("No running SQL warehouses found. Selecting the first available (stopped) warehouse.")
        else:
            # This case should ideally be caught by `if not all_warehouses:` but as a fallback
            print("No running or stopped SQL warehouses found.")

        if first_available_warehouse:
            warehouse_id = first_available_warehouse.id
            warehouse_name = first_available_warehouse.name
            warehouse_state = first_available_warehouse.state.value

            print("\n--- First Available SQL Warehouse Details ---")
            print(f"ID: {warehouse_id}")
        else:
            print("Could not determine an available SQL warehouse ID.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure your Databricks authentication is correctly configured and you have permissions to list SQL warehouses.")

In [0]:
#deploy app
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import apps
from databricks.sdk.service.apps import App, AppResource, AppDeployment, AppResourceSqlWarehouse, AppResourceSqlWarehouseSqlWarehousePermission
from pathlib import Path

w = WorkspaceClient()

current_notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
# Use pathlib to handle path manipulation robustly
current_notebook_dir = Path(current_notebook_path).parent.parent
full_app_notebook_path = (current_notebook_dir / 'src/').resolve()

# Convert the Path object back to a string for the API
directory_path = str(full_app_notebook_path)

app_name = f"fe-demo-{schema_name.lower().replace('_', '-')}"

warehouse = AppResourceSqlWarehouse(
    id=warehouse_id,
    permission=AppResourceSqlWarehouseSqlWarehousePermission.CAN_USE
)

warehouse_resource = AppResource(name="sql-warehouse", sql_warehouse=warehouse)

evaluation_app = App(name=app_name, 
              description="Demo Iron Ore Processing App", 
              default_source_code_path=directory_path,
              resources=[warehouse_resource],
              user_api_scopes=['sql']
              )

app_details = w.apps.create_and_wait(app=evaluation_app)
print(app_details)


In [0]:

deployment = AppDeployment(
  source_code_path=f"/Workspace{directory_path}"
)

app_details = w.apps.deploy_and_wait(app_name=app_name, app_deployment=deployment)

In [0]:
import json
#Let's access the application
app_url = w.apps.get(name=app_name).url

content = {
  "app_url" : app_url
}

with open("object_urls.json", "r") as file:
    data = json.load(file)

data.update(content)

with open("object_urls.json", "w") as file:
    json.dump(data, file)